In [ ]:
# Inventory Management System using Factory Design Pattern
import sqlite3
from datetime import datetime

DB_NAME = "inventory_notebook.db"

# ---------- Product classes & Factory ----------

class Product:
    """Generic product object (used by the factory and for DB insertion)."""
    def __init__(self, name, category="General", quantity=0, price=0.0):
        self.name = str(name)
        self.category = str(category)
        self.quantity = int(quantity)
        self.price = float(price)
        self.created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def to_tuple(self):
        return (self.name, self.category, self.quantity, self.price, self.created_at)

    def __repr__(self):
        return f"Product(name={self.name!r}, category={self.category!r}, qty={self.quantity}, price={self.price})"

class Electronics(Product):
    def __init__(self, name, quantity=0, price=0.0, warranty_months=12):
        super().__init__(name, "Electronics", quantity, price)
        self.warranty_months = int(warranty_months)

class Grocery(Product):
    def __init__(self, name, quantity=0, price=0.0, perishable=True):
        super().__init__(name, "Grocery", quantity, price)
        self.perishable = bool(perishable)

class Clothing(Product):
    def __init__(self, name, quantity=0, price=0.0, size="M"):
        super().__init__(name, "Clothing", quantity, price)
        self.size = str(size)

class ProductFactory:
    """Simple factory: create(product_category, name, quantity, price, **extras)"""
    @staticmethod
    def create(category, name, quantity=0, price=0.0, **extras):
        cat = (category or "General").strip().lower()
        if cat in ("electronics", "electronic"):
            return Electronics(name, quantity=quantity, price=price, warranty_months=extras.get("warranty_months", 12))
        if cat in ("grocery", "food"):
            return Grocery(name, quantity=quantity, price=price, perishable=extras.get("perishable", True))
        if cat in ("clothing", "apparel", "garment"):
            return Clothing(name, quantity=quantity, price=price, size=extras.get("size", "M"))
        return Product(name, category=category or "General", quantity=quantity, price=price)

# ---------- Database helper functions ----------

def get_conn():
    """Return a sqlite3.Connection using the notebook DB file."""
    return sqlite3.connect(DB_NAME)

def init_db():
    """Create products table if missing."""
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                category TEXT,
                quantity INTEGER,
                price REAL,
                created_at TEXT
            )
        """)
        conn.commit()

def add_product_to_db(product):
    """Insert a Product instance into DB. Returns new row id."""
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute(
            "INSERT INTO products (name, category, quantity, price, created_at) VALUES (?, ?, ?, ?, ?)",
            product.to_tuple()
        )
        conn.commit()
        return cur.lastrowid

def fetch_products(limit=100, order_by="id"):
    """Return list of rows (tuples) from products table."""
    q = f"SELECT id, name, category, quantity, price, created_at FROM products ORDER BY {order_by} ASC LIMIT ?"
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute(q, (limit,))
        return cur.fetchall()

def search_products(keyword):
    """Search by name or category (simple LIKE). Returns rows."""
    kw = f"%{keyword}%"
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute(
            "SELECT id, name, category, quantity, price, created_at FROM products WHERE name LIKE ? OR category LIKE ? ORDER BY id ASC",
            (kw, kw)
        )
        return cur.fetchall()

def get_product_by_id(pid):
    """Return single row or None."""
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute("SELECT id, name, category, quantity, price, created_at FROM products WHERE id = ?", (pid,))
        return cur.fetchone()

def update_product(pid, **fields):
    """
    Update allowed fields for a product.
    Allowed: name, category, quantity, price
    Returns True if row updated, False otherwise.
    """
    allowed = {"name", "category", "quantity", "price"}
    updates = {k: fields[k] for k in fields if k in allowed}
    if not updates:
        return False
    cols = ", ".join(f"{k} = ?" for k in updates.keys())
    vals = list(updates.values()) + [pid]
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute(f"UPDATE products SET {cols} WHERE id = ?", vals)
        conn.commit()
        return cur.rowcount > 0

def delete_product(pid):
    """Delete product by id. Returns True if deleted."""
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute("DELETE FROM products WHERE id = ?", (pid,))
        conn.commit()
        return cur.rowcount > 0

def adjust_quantity(pid, delta):
    """
    Add (positive) or remove (negative) quantity.
    Returns (True, new_qty) on success, (False, message) on failure.
    """
    row = get_product_by_id(pid)
    if not row:
        return False, "Product not found"
    current = int(row[3])
    new_qty = current + int(delta)
    if new_qty < 0:
        return False, "Quantity cannot be negative"
    ok = update_product(pid, quantity=new_qty)
    return (True, new_qty) if ok else (False, "Update failed")

# Initialize DB on import/run
init_db()


In [22]:
# create some products using factory
p1 = ProductFactory.create("Electronics", "Laptop", quantity=5, price=45000)
p2 = ProductFactory.create("Grocery", "Milk (1L)", quantity=50, price=45.0)
p3 = ProductFactory.create("Clothing", "T-Shirt", quantity=30, price=299.0)

# add to DB
id1 = add_product_to_db(p1)
id2 = add_product_to_db(p2)
id3 = add_product_to_db(p3)
id1, id2, id3


(1, 2, 3)

In [23]:
rows = fetch_products(limit=20)
for r in rows:
    print(r)


(1, 'Laptop', 'Electronics', 5, 45000.0, '2025-11-26 22:55:20')
(2, 'Milk (1L)', 'Grocery', 50, 45.0, '2025-11-26 22:55:20')
(3, 'T-Shirt', 'Clothing', 30, 299.0, '2025-11-26 22:55:20')


In [24]:
search_products("lap")     # matches Laptop
search_products("groc")    # matches Grocery category


[(2, 'Milk (1L)', 'Grocery', 50, 45.0, '2025-11-26 22:55:20')]

In [ ]:
# update price and name
update_product(id1, price=43000.0, name="Laptop Pro")

# increase stock by 10
ok, result = adjust_quantity(id2, 10)
print(ok, result)

# delete a product
delete_product(id3)


True 60


True